In [6]:
'''
    Definition of terms:
        [a] Transcranial Electrical Simulation
        [b] Transcranial Magnetic Simulation
    
    Given .nii file
    Create a mesh and .msh file
    Add tDCS (Transcranial Direct Current Simulation) - non-invasive brain stimulation 
        that uses electrical currents to simulate specific parts of the brain
        -- Add tDCS postlist
    Required files:
        [1] Coil position 'Magstim_70mm_Fig8.nii'
                More info: https://simnibs.github.io/simnibs/build/html/_downloads/2c1fbb815968be33435802061a411624/Deng_Brain_Stim_2013_docu.pdf
                
        [2] MSH files

    TODO:
        [1] Create gmsh files (.msh) from T1W MRI images, UQ results
        [2] Calculate dA/dt ()
        [3] Get the conductivity(ROI) for the conductivity 
'''

In [6]:
import gmsh
import math
import os
import sys

gmsh.initialize()

def createGeometryAndMesh():
    # Clear all models and merge an STL mesh that we would like to remesh (from
    # the parent directory):
    gmsh.clear()
    sample = None
    path = os.path.dirname(os.path.abspath(sample))
    gmsh.merge(os.path.join(path, os.pardir, 't13_data.stl'))
    # We first classify ("color") the surfaces by splitting the original surface
    # along sharp geometrical features. This will create new discrete surfaces,
    # curves and points.
    # Angle between two triangles above which an edge is considered as sharp,
    # retrieved from the ONELAB database (see below):
    angle = gmsh.onelab.getNumber('Parameters/Angle for surface detection')[0]
    # For complex geometries, patches can be too complex, too elongated or too
    # large to be parametrized; setting the following option will force the
    # creation of patches that are amenable to reparametrization:
    forceParametrizablePatches = gmsh.onelab.getNumber(
        'Parameters/Create surfaces guaranteed to be parametrizable')[0]
    # For open surfaces include the boundary edges in the classification
    # process:
    includeBoundary = True
    curveAngle = 180
    gmsh.model.mesh.classifySurfaces(angle * math.pi / 180., includeBoundary,
                                     forceParametrizablePatches,
                                     curveAngle * math.pi / 180.)

    # Create a geometry for all the discrete curves and surfaces in the mesh, by
    # computing a parametrization for each one
    gmsh.model.mesh.createGeometry()

    # Create a volume from all the surfaces
    s = gmsh.model.getEntities(2)
    l = gmsh.model.geo.addSurfaceLoop([e[1] for e in s])
    gmsh.model.geo.addVolume([l])

    gmsh.model.geo.synchronize()

    # We specify element sizes imposed by a size field, just because we can :-)
    f = gmsh.model.mesh.field.add("MathEval")
    if gmsh.onelab.getNumber('Parameters/Apply funny mesh size field?')[0]:
        gmsh.model.mesh.field.setString(f, "F", "2*Sin((x+y)/5) + 3")
    else:
        gmsh.model.mesh.field.setString(f, "F", "4")
    gmsh.model.mesh.field.setAsBackgroundMesh(f)

    gmsh.model.mesh.generate(3)
    gmsh.write('t13.msh')
# Create ONELAB parameters with remeshing options:
gmsh.onelab.set("""[
  {
    "type":"number",
    "name":"Parameters/Angle for surface detection",
    "values":[40],
    "min":20,
    "max":120,
    "step":1
  },
  {
    "type":"number",
    "name":"Parameters/Create surfaces guaranteed to be parametrizable",
    "values":[0],
    "choices":[0, 1]
  },
  {
    "type":"number",
    "name":"Parameters/Apply funny mesh size field?",
    "values":[0],
    "choices":[0, 1]
  }
]""")

# Create the geometry and mesh it:
# createGeometryAndMesh()

# Launch the GUI and handle the "check" event to recreate the geometry and mesh
# with new parameters if necessary:
# def checkForEvent():
#     action = gmsh.onelab.getString("ONELAB/Action")
#     if len(action) and action[0] == "check":
#         gmsh.onelab.setString("ONELAB/Action", [""])
#         createGeometryAndMesh()
#         gmsh.graphics.draw()
#     return True

# if "-nopopup" not in sys.argv:
#     gmsh.fltk.initialize()
#     while gmsh.fltk.isAvailable() and checkForEvent():
#         gmsh.fltk.wait()

# gmsh.finalize()

In [17]:
from simnibs import sim_struct, run_simnibs

''' General Settings '''
# Initalize a session
s = sim_struct.SESSION()
# Name of head mesh
s.fnamehead = 'ernie.msh'
# Output folder
s.pathfem = 'tutorial/'

''' TMS simulations '''
# Initialize a list of TMS simulations
tmslist = s.add_tmslist()
# Select coil
tmslist.fnamecoil = 'Magstim_70mm_Fig8.nii.gz'


''' First coil position '''
# Initialize a coil position
pos = tmslist.add_position()
# Select coil centre
pos.centre = 'C1'
# Select coil direction
pos.pos_ydir = 'CP1'

''' Second coil position '''
# Add another position
pos_superior = tmslist.add_position()
# Centred at C1
pos_superior.centre = 'C1'
# Pointing towards Cz
pos_superior.pos_ydir = 'Cz'

''' tDCS Simulation '''
tdcslist = s.add_tdcslist()
# Set currents
tdcslist.currents = [-1e-3, 1e-3]

''' Define cathode '''
# Initialize the cathode
cathode = tdcslist.add_electrode()
# Connect electrode to first channel (-1e-3 mA, cathode)
cathode.channelnr = 1
# Electrode dimension
cathode.dimensions = [50, 70]
# Rectangular shape
cathode.shape = 'rect'
# 5mm thickness
cathode.thickness = 5
# Electrode Position
cathode.centre = 'C3'
# Electrode direction
cathode.pos_ydir = 'Cz'

''' Define anode '''
# Add another electrode
anode = tdcslist.add_electrode()
# Assign it to the second channel
anode.channelnr = 2
# Electrode diameter
anode.dimensions = [30, 30]
# Electrode shape
anode.shape = 'ellipse'
# 5mm thickness
anode.thickness = 5
# Electrode position
anode.centre = 'C4'

''' Run Simulations '''
run_simnibs(s)


In [ ]:
import vtk
import SimpleITK as sitk
import numpy as np

def generate_mesh(filename_nii, filename_mesh, save = False):

    image = sitk.GetArrayFromImage(sitk.ReadImage(filename_nii))
    label = np.unique(image)

    reader = vtk.vtkNIFTIImageReader()
    reader.SetFileName(filename_nii)
    reader.Update()

    for lbl, idx in enumerate(label):
        print(f'Index {idx}')
        surf = vtk.vtkDiscreteMarchingCubes()
        surf.SetInputConnection(reader.GetOutputPort())
        surf.SetValue(0, lbl)
        surf.Update()

        smoother = vtk.vtkWindowedSincPolyDataFilter('False')
        if vtk.VTK_MAJOR_VERSION <= 5:
            smoother.SetInput(surf.GetOutput())
        else:
            smoother.SetInputConnection(surf.GetOutputPort())
        smoother.SetNumberOfIterations(30)
        smoother.NonManifoldSmoothingOn()
        smoother.NormalizeCoordinatesOn()
        smoother.GenerateErrorScalarsOn()
        smoother.Update()

        if save is True:
            writer = vtk.vtkSTLWriter()
            writer.SetInputConnection(smoother.GetOutputPort())
            writer.SetFileTypeToASCII()
            writer.SetFileName(filename_mesh)
            writer.Write()


dir = 'C:/Users/Renan/Desktop/dsai-thesis/train/labels/T1W-012.nii'
tar = 'C:/Users/Renan/Desktop/dsai-thesis/stl'
generate_mesh(dir, tar)

def loadStl(fname):
    """Load the given STL file, and return a vtkPolyData object for it."""
    reader = vtk.vtkSTLReader()
    reader.SetFileName(fname)
    reader.Update()
    polydata = reader.GetOutput()
    return polydata

def polyDataToActor(polydata):
    mapper = vtk.vtkPolyDataMapper()
    if vtk.VTK_MAJOR_VERSION <= 5:
        mapper.SetInput(polydata)
    else:
        mapper.SetInputConnection(polydata.GetProducerPort())
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(0.5, 0.5, 1.0)
    return actor

def render():
    ren = vtk.vtkRenderer()
    renWin = vtk.vtkRenderWindow()
    renWin.AddRenderer(ren)
    iren = vtk.vtkRenderWindowInteractor()
    iren.SetRenderWindow(renWin)
    style = vtk.vtkInteractorStyleTrackballCamera()
    iren.SetInteractorStyle(style)

    stlFilename = 'C:/Users/Renan/Desktop/dsai-thesis/3dprint.stl'
    polydata = loadStl(stlFilename)
    ren.SetBackground(0.1, 0.1, 0.1)
    iren.Initialize()
    renWin.Render()

In [3]:
import vtkmodules.vtkInteractionStyle
# noinspection PyUnresolvedReferences
import vtkmodules.vtkRenderingOpenGL2
from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkFiltersSources import vtkCylinderSource
from vtkmodules.vtkRenderingCore import (
    vtkActor,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderWindowInteractor,
    vtkRenderer
)


def main():
    colors = vtkNamedColors()
    # Set the background color.
    bkg = map(lambda x: x / 255.0, [26, 51, 102, 255])
    colors.SetColor("BkgColor", *bkg)

    # This creates a polygonal cylinder model with eight circumferential
    # facets.
    cylinder = vtkCylinderSource()
    cylinder.SetResolution(10)

    # The mapper is responsible for pushing the geometry into the graphics
    # library. It may also do color mapping, if scalars or other
    # attributes are defined.
    cylinderMapper = vtkPolyDataMapper()
    cylinderMapper.SetInputConnection(cylinder.GetOutputPort())

    # The actor is a grouping mechanism: besides the geometry (mapper), it
    # also has a property, transformation matrix, and/or texture map.
    # Here we set its color and rotate it -22.5 degrees.
    cylinderActor = vtkActor()
    cylinderActor.SetMapper(cylinderMapper)
    cylinderActor.GetProperty().SetColor(colors.GetColor3d("Tomato"))
    cylinderActor.RotateX(30.0)
    cylinderActor.RotateY(-45.0)

    # Create the graphics structure. The renderer renders into the render
    # window. The render window interactor captures mouse events and will
    # perform appropriate camera or actor manipulation depending on the
    # nature of the events.
    ren = vtkRenderer()
    renWin = vtkRenderWindow()
    renWin.AddRenderer(ren)
    iren = vtkRenderWindowInteractor()
    iren.SetRenderWindow(renWin)

    # Add the actors to the renderer, set the background and size
    ren.AddActor(cylinderActor)
    ren.SetBackground(colors.GetColor3d("BkgColor"))
    renWin.SetSize(300, 300)
    renWin.SetWindowName('CylinderExample')

    # This allows the interactor to initalize itself. It has to be
    # called before an event loop.
    iren.Initialize()

    # We'll zoom in a little by accessing the camera and invoking a "Zoom"
    # method on it.
    ren.ResetCamera()
    ren.GetActiveCamera().Zoom(1.5)
    renWin.Render()

    # Start the event loop.
    iren.Start()


if __name__ == '__main__':
    main()